## LangChain Tool
Tool : llm 에이전트가 수행할 수 있는 외부기능이나 api를 말함(날씨정보나 웹검색, 계산기)  
사용하는 이유 : gpt는 최신정보에 접근할 수 없다.  
구조 : 이름 설명 함수 실행 로직
툴 호출 방식 : GPT-4 모델들은 OpenAI의 함수 호출 기능을 통해 툴을 직접 호출할 수 있다.


In [1]:
%pip install langchain-openai langchain-community python-dotenv openai

Note: you may need to restart the kernel to use updated packages.


In [2]:
# .env 로드
from dotenv import load_dotenv
import os
load_dotenv()

print(os.environ["OPEN_WEATHER_MAP"][:5])
print(os.environ["NEWSAPI_API_KEY"][:5])

de3d1
2e346


In [3]:
import requests
from pydantic import BaseModel, Field

# 입력 스키마 정의
class WeatherInput(BaseModel):
    city: str = Field(description="날씨를 조회할 도시 이름 (영문)")

def get_weather(city: str) -> str:
    """주어진 도시의 현재 날씨를 반환합니다."""
    api_key = os.getenv("OPEN_WEATHER_MAP")
    url = "http://api.openweathermap.org/data/2.5/weather"
    params = {
        "q": city,
        "appid": api_key,
        "units": "metric",      # 섭씨 온도
        "lang": "kr"            # 한국어 응답 (설명이 한국어로 오도록)
    }
    response = requests.get(url, params=params)
    data = response.json()
    # API 응답에 따른 처리
    if data.get("cod") != 200:
        # 도시 정보를 찾지 못한 경우
        return f"'{city}'의 날씨 정보를 찾을 수 없습니다."
    # 필요한 정보 파싱
    temp = data["main"]["temp"]
    desc = data["weather"][0]["description"]
    city_name = data["name"]  # 응답에 있는 도시 이름 (영문일 수 있음)
    return f"{city_name}의 현재 기온은 {temp}℃, 날씨는 {desc}입니다."

In [4]:
# # pydantic 모델을 사용하여 입력값 검증
# class WeatherInput(BaseModel):
#     city: str = Field(description="날씨를 조회할 도시 이름(영문)")

# class MyClass:
#     def __init__(self, ab:str):
#         self.city = ab

# MyClass(ab=100) # 사용자가 의도한 문자열이 전달되지 않아도 python은 타입에러를 발생시키지 않음
# result = WeatherInput(city=100) # 문자열이 아니면 pydantic이 에러를 발생시킴
# result.city

In [5]:
get_weather("seoul")

'Seoul의 현재 기온은 24.76℃, 날씨는 약간의 구름이 낀 하늘입니다.'

## 뉴스검색

In [6]:
from pydantic import BaseModel, Field

class NewsInput(BaseModel):
    keyword: str = Field(description="뉴스를 조회할 키워드")
    
def get_news(keyword:str) -> str:
    import requests
    api_key = os.getenv("NEWSAPI_API_KEY")
    url = "https://newsapi.org/v2/everything"
    params = {
        "q": keyword,
        "apiKey": api_key,
        "language": "ko",
        "sortBy": "publishedAt",  #최신뉴스
        "pageSize": 1  # 최근 1건
    }
    response = requests.get(url, params=params)
    data = response.json()    
    articles = data.get('articles')
    if not articles:
        return "해당 키워드에 대한 뉴스가 없습니다."
    
    # 가장 첫번째 뉴스선택
    top_news = articles[0]
    title = top_news.get('title', '제목 없음')
    source = top_news.get('source', {}).get('name', '출처 없음')
    return f'{keyword}에 대한 최신 뉴스: "{title}" (출처: {source})'

In [7]:
print(get_news("취업"))

취업에 대한 최신 뉴스: "울산시, '광역 비자 시범 대상지'로 최종 선정" (출처: Ohmynews.com)


# LangChain 객체로 변환
get_weather, get_news 함수를 Tool로 등록

In [8]:
from langchain_core.runnables import RunnableLambda

# 날씨 함수 runnable -> tool 변환
weather_runnable = RunnableLambda(lambda input: get_weather(input["city"]))
weather_tool = weather_runnable.as_tool(
    name="get_weather",
    description="도시 이름(영문)을 입력하면 현재 날씨를 알려주는 도구",
    args_schema=WeatherInput
)

# 뉴스 함수 runnable -> tool 변환
news_runnable = RunnableLambda(lambda input: get_news(input["keyword"]))
news_tool = news_runnable.as_tool(
    name="get_news",
    description="키워드를 입력하면 관련된 최신 뉴스 제목을 알려주는 도구",
    args_schema=NewsInput
)

print(f"Tool 이름: {weather_tool.name}, 설명: {weather_tool.description}")
print(f"Tool 이름: {news_tool.name}, 설명: {news_tool.description}")

C:\Users\bokji\AppData\Local\Temp\ipykernel_13244\1572052777.py:5: LangChainBetaWarning: This API is in beta and may change in the future.
  weather_tool = weather_runnable.as_tool(


Tool 이름: get_weather, 설명: 도시 이름(영문)을 입력하면 현재 날씨를 알려주는 도구
Tool 이름: get_news, 설명: 키워드를 입력하면 관련된 최신 뉴스 제목을 알려주는 도구


In [9]:
# 툴없이 
from langchain_openai import ChatOpenAI
# 프롬프트 템플릿
from langchain_core.prompts import ChatPromptTemplate
# 출력 파서
from langchain_core.output_parsers import StrOutputParser
user_question = '서울 날씨 알려줘'
# 사용자 질문을 처리하는 프롬프트 템플릿
user_prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 날씨와 뉴스 정보를 제공하는 AI입니다."),
    ("human", "{question}")
])
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.0)
chain = user_prompt | llm | StrOutputParser()
# 사용자 질문을 처리하는 체인 실행
result = chain.invoke({"question": user_question})
print(result)

현재 서울의 날씨는 실시간으로 확인할 수 없지만, 일반적으로 10월의 서울은 가을 날씨로 쌀쌀하고 맑은 날이 많습니다. 평균 기온은 약 10도에서 20도 사이입니다. 구체적인 날씨 정보는 기상청 웹사이트나 날씨 앱을 통해 확인하시기 바랍니다. 추가로 궁금한 점이 있으면 말씀해 주세요!


LLM 최종 답변 생성 요청

In [10]:
from langchain.schema import AIMessage, HumanMessage, SystemMessage
# 프롬프트구성 : 시스템메시지 + 사용자 질문 + 결과를 맥락으로 제공
weather_info = get_weather('seoul')
system_prompt = "당신은 유용한 AI 어시스턴트입니다. 사용자 질문에 맞게 제공된 정보를 활용해 답변하세요"
tool_info_prompt = f'도구가 제공한 추가 정보:{weather_info}'
ask_prompt = '위 정보를 참고해서 사용자 질문에 답변하세요'
messages = [
    SystemMessage(content=system_prompt),
    HumanMessage(content=user_question),
    AIMessage(content=tool_info_prompt),  #도구 결과를 마치 ai의 답변처럼 추가가
    HumanMessage(content=ask_prompt)
]
result = llm(messages)   # LLM에 메시지 전달
print(result.content)

C:\Users\bokji\AppData\Local\Temp\ipykernel_13244\2540460585.py:13: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = llm(messages)   # LLM에 메시지 전달


현재 서울의 날씨는 약간의 구름이 낀 상태이며, 기온은 약 24.76℃입니다. 추가적인 날씨 정보가 필요하시면 말씀해 주세요!


여러 tool을 연결하도록 agent 구성

In [11]:
# 에이전트를 생성하고 등록
from langchain.agents import initialize_agent, AgentType

# 앞서 만든 Tool 객체 리스트 준비
tools = [weather_tool, news_tool]

# OpenAI 함수 기반 에이전트 생성
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.OPENAI_FUNCTIONS,
    verbose=True  # intermediate 단계 로그를 출력하여 확인
)

C:\Users\bokji\AppData\Local\Temp\ipykernel_13244\1215030893.py:8: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(


In [12]:
## 에이전트 실행 : 단일 툴 선택

query1 = "서울의 날씨와 AI 관련 최신 뉴스를 알려줘."
result1 = agent.run(query1)
print("Agent Result 1:", result1)

C:\Users\bokji\AppData\Local\Temp\ipykernel_13244\1339179620.py:4: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result1 = agent.run(query1)




> Entering new AgentExecutor chain...

Invoking: `get_weather` with `{'city': 'Seoul'}`


Seoul의 현재 기온은 24.76℃, 날씨는 약간의 구름이 낀 하늘입니다.
Invoking: `get_news` with `{'keyword': 'AI'}`


AI에 대한 최신 뉴스: "제국의 귀환: 세르게이 브린, “첫 AGI는 제미나이”" (출처: Slownews.kr)서울의 현재 날씨는 약간의 구름이 낀 하늘이며, 기온은 24.76℃입니다.

AI 관련 최신 뉴스로는 "제국의 귀환: 세르게이 브린, “첫 AGI는 제미나이”"라는 제목의 기사가 있습니다. (출처: Slownews.kr)

> Finished chain.
Agent Result 1: 서울의 현재 날씨는 약간의 구름이 낀 하늘이며, 기온은 24.76℃입니다.

AI 관련 최신 뉴스로는 "제국의 귀환: 세르게이 브린, “첫 AGI는 제미나이”"라는 제목의 기사가 있습니다. (출처: Slownews.kr)


## 검색

In [13]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

load_dotenv()
# https://custombooks.com/chatgpt-prompts?utm_source=chatgpt.com
llm = ChatOpenAI(model='gpt-4o-mini', temperature=0)
prompt_template = ChatPromptTemplate.from_messages([
    ("system", '''
        Want assistance provided by qualified individuals enabled with experience on understanding charts using technical analysis tools while interpreting macroeconomic environment prevailing across world consequently assisting customers acquire long term advantages requires clear verdicts therefore seeking same through informed predictions written down precisely! First statement contains following content- “Can you tell us what future stock market looks like based upon current conditions ?.
     '''),
    ("user", "{company}주식에 투자해도 될까요? 마크다운 형식으로 투자 보고서를 한글로 작성해 주세요"),
])

In [14]:
from langchain_core.output_parsers import StrOutputParser  
# 프롬프트 체인 정의
chain = prompt_template | llm | StrOutputParser()
# 프롬프트 체인 실행
result = chain.invoke({"company": "삼성전자"})
print(result)


# 삼성전자 투자 보고서

## 1. 회사 개요
삼성전자는 전자제품, 반도체, 통신 장비 등 다양한 분야에서 세계적인 기업으로 자리 잡고 있습니다. 특히 반도체 분야에서의 강력한 입지와 스마트폰 시장에서의 경쟁력으로 잘 알려져 있습니다.

## 2. 현재 시장 상황
### 2.1. 글로벌 경제
- **금리 인상**: 주요 국가들의 금리 인상이 지속되고 있어, 자금 조달 비용이 증가하고 있습니다.
- **인플레이션**: 전 세계적으로 인플레이션 압력이 높아지고 있으며, 이는 소비자 지출에 영향을 미칠 수 있습니다.

### 2.2. 반도체 산업
- **수요 증가**: AI, IoT, 5G 등 기술 발전으로 반도체 수요가 증가하고 있습니다.
- **공급망 문제**: 여전히 공급망 문제가 존재하지만, 점차 회복세를 보이고 있습니다.

## 3. 삼성전자의 재무 상태
- **매출 성장**: 최근 몇 년간 매출이 지속적으로 성장하고 있으며, 특히 반도체 부문에서의 성장이 두드러집니다.
- **부채 비율**: 안정적인 부채 비율을 유지하고 있어 재무 건전성이 높습니다.

## 4. 기술적 분석
- **주가 추세**: 최근 주가는 상승세를 보이고 있으며, 주요 지지선과 저항선을 분석할 필요가 있습니다.
- **지표 분석**: RSI, MACD 등 기술적 지표를 통해 과매도 또는 과매수 상태를 확인할 수 있습니다.

## 5. 투자 결론
삼성전자는 안정적인 재무 상태와 성장 가능성을 가지고 있으며, 반도체 산업의 수요 증가로 인해 긍정적인 전망을 가지고 있습니다. 그러나 글로벌 경제의 불확실성과 금리 인상 등의 리스크를 고려해야 합니다.

### **추천**
- **매수**: 장기 투자 관점에서 긍정적인 신호가 보이며, 적절한 가격대에서 매수하는 것이 좋습니다.
- **모니터링**: 시장 상황과 기술적 지표를 지속적으로 모니터링하여 투자 결정을 조정해야 합니다.

## 6. 참고 자료
- 삼성전자 공식 웹사이트
- 금융 뉴스 및 리서치 보고서
- 기술적 분석 도구 및 차트


In [15]:
import yfinance as yf
company = yf.Ticker("005930.KS")  # 삼성전자
# 주식 정보 가져오기    
company.info

{'address1': '129 Samsung-Ro',
 'address2': 'Maetan-3dong Yeongtong-gu',
 'city': 'Suwon',
 'country': 'South Korea',
 'website': 'https://www.samsung.com',
 'industry': 'Consumer Electronics',
 'industryKey': 'consumer-electronics',
 'industryDisp': 'Consumer Electronics',
 'sector': 'Technology',
 'sectorKey': 'technology',
 'sectorDisp': 'Technology',
 'longBusinessSummary': 'Samsung Electronics Co., Ltd. engages in the consumer electronics, information technology and mobile communications, device solutions businesses, and R&D Centers worldwide. It operates in four divisions such as DX, DS, SDC and Harman. The company offers smartphones, tablets, audio sounds, watches, switches, and accessories; TVs, and sound devices; appliances, including refrigerators, washing machines and dryers, vacuum cleaners, cooking appliances, dishwashers, air conditioners, and air purifiers; monitors and memory storage products; displays, and smart and LED signages; and other accessories. It also engages 

In [16]:
basic_info = {
    "회사명": company.info.get("longName", "정보 없음"),
    "산업": company.info.get("industry", "정보 없음"),
    "부문": company.info.get("sector", "정보 없음"),
    "시가총액": company.info.get("marketCap", "정보 없음"),
    "총 발행 주식 수": company.info.get("sharesOutstanding", "정보 없음"),
    "본사 위치": company.info.get("country", "정보 없음"),
    "웹사이트": company.info.get("website", "정보 없음"),
}

# 기본 정보 출력
for key, value in basic_info.items():
    print(f"{key}: {value}")

회사명: Samsung Electronics Co., Ltd.
산업: Consumer Electronics
부문: Technology
시가총액: 355089441095680
총 발행 주식 수: 5919640064
본사 위치: South Korea
웹사이트: https://www.samsung.com


In [ ]:
income_statement = company.financials
# 재무제표 출력
income_statement

,2024-12-31,2023-12-31,2022-12-31,2021-12-31,2020-12-31
Tax Effect Of Unusual Items,-7643351884.3752,-22864080000.0,-25405600000.0,-13557566210.000643,NaN
Tax Rate For Calcs,0.082025,0.24,0.275,0.251995,NaN
Normalized EBITDA,81157657000000.0,50698344000000.0,86403532000000.0,88084529000000.0,NaN
Total Unusual Items,-93183000000.0,-95267000000.0,-92384000000.0,-53801000000.0,NaN
Total Unusual Items Excluding Goodwill,-93183000000.0,-95267000000.0,-92384000000.0,-53801000000.0,NaN
Net Income From Continuing Operation Net Minority Interest,33621363000000.0,14473401000000.0,54730018000000.0,39243791000000.0,NaN
Reconciled Depreciation,42630822000000.0,38666559000000.0,39107659000000.0,34247361000000.0,NaN
Reconciled Cost Of Revenue,186562268000000.0,180388580000000.0,190041770000000.0,166411342000000.0,NaN
EBITDA,81064474000000.0,50603077000000.0,86311148000000.0,88030728000000.0,NaN
EBIT,38433652000000.0,11936518000000.0,47203489000000.0,53783367000000.0,NaN


In [ ]:
balance_sheet = company.balance_sheet
# 대차대조표 출력
balance_sheet

,2024-12-31,2023-12-31,2022-12-31,2021-12-31,2020-12-31
Treasury Shares Number,NaN,0.0,0.0,0.0,0.0
Preferred Shares Number,822886700.0,822886700.0,822886700.0,822886700.0,NaN
Ordinary Shares Number,6792669250.0,5969782550.0,5969782550.0,5969782550.0,NaN
Share Issued,6792669250.0,5969782550.0,5969782550.0,5969782550.0,NaN
Total Debt,19330184000000.0,12685944000000.0,10333242000000.0,18392149000000.0,NaN
...,...,...,...,...,...
Gross Accounts Receivable,44044073000000.0,37002849000000.0,36033784000000.0,41024295000000.0,NaN
Cash Cash Equivalents And Short Term Investments,112651790000000.0,92407210000000.0,115227286000000.0,124150192000000.0,NaN
Other Short Term Investments,58946211000000.0,23326317000000.0,65546576000000.0,85118777000000.0,NaN
Cash And Cash Equivalents,53705579000000.0,69080893000000.0,49680710000000.0,39031415000000.0,NaN


In [19]:
# 현금흐름표 
cash_flow = company.cashflow
cash_flow

,2024-12-31,2023-12-31,2022-12-31,2021-12-31,2020-12-31
Free Cash Flow,19240982000000.0,-16396740000000.0,9054614000000.0,15276427000000.0,NaN
Repurchase Of Capital Stock,-1811775000000.0,NaN,NaN,NaN,NaN
Repayment Of Debt,-1364508000000.0,-1219579000000.0,-9847614000000.0,-3511692000000.0,NaN
Issuance Of Debt,6276300000000.0,2500112000000.0,271997000000.0,58279000000.0,NaN
Capital Expenditure,-53741639000000.0,-60534167000000.0,-53126732000000.0,-49829021000000.0,NaN
...,...,...,...,...,...
Depreciation,39649982000000.0,35532411000000.0,35952098000000.0,31285209000000.0,NaN
Pension And Employee Benefit Expense,1571338000000.0,1157422000000.0,1440099000000.0,1360344000000.0,NaN
Gain Loss On Investment Securities,-134952000000.0,-164203000000.0,-414601000000.0,-135840000000.0,NaN
Gain Loss On Sale Of PPE,42371000000.0,-18864000000.0,-97867000000.0,-264814000000.0,NaN


In [20]:
pe_ratio = company.info.get('forwardPE')
pb_ratio = company.info.get('priceToBook')
eps = company.info.get('trailingEps')
roe = company.info.get('returnOnEquity')
current_volume = company.info.get('volume')
current_volume

13348405

In [21]:
prompt = ChatPromptTemplate.from_messages([
    ("system", """
        Want assistance provided by qualified individuals enabled with experience on understanding charts 
        using technical analysis tools while interpreting macroeconomic environment prevailing across world 
        consequently assisting customers acquire long term advantages requires clear verdicts therefore 
        seeking same through informed predictions written down precisely! First statement contains 
        following content- "Can you tell us what future stock market looks like based upon current conditions ?".
    """),
    ("user", """
        {company} 주식에 투자해도 될까요?
        아래의 기본정보, 재무제표를 참고해 마크다운 형식의 투자보고서를 한글로 작성해 주세여.

        기본정보:
        {basic_info}

        재무제표:
        {financial_statement}

     """)
])

In [23]:
from langchain_core.output_parsers import StrOutputParser
from stock_info import Stock
chain = prompt | llm | StrOutputParser()
company = 'nvidia'
symbol = 'NVDA'
stock = Stock(symbol)
result = chain.invoke({
    "company": company,
    "basic_info": stock.get_basic_info(),
    "financial_statement": stock.get_financial_statement()
})
# md파일로 저장
with open(f"{company}_investment_report.md", "w", encoding="utf-8") as f:
    f.write(result)

In [24]:
import pandas as pd
df = pd.read_csv('nasdaq_screener.csv',na_filter=False)
# df['id'] = df['Symbol'].str.strip().replace(r'\^', '_', regex=True)
df['id'] = df['Symbol'].str.strip().replace(r'[/^]', '_', regex=True)

In [25]:
result_d = df.to_dict(orient='records')
result_d[0]

{'Symbol': 'A',
 'Name': 'Agilent Technologies Inc. Common Stock',
 'Last Sale': '$108.53',
 'Net Change': -1.15,
 '% Change': '-1.049%',
 'Market Cap': '30942197876.00',
 'Country': 'United States',
 'IPO Year': '1999',
 'Volume': 1690973,
 'Sector': 'Industrials',
 'Industry': 'Biotechnology: Laboratory Analytical Instruments',
 'id': 'A'}

In [26]:
from dotenv import load_dotenv
load_dotenv()
import meilisearch

In [27]:
import os
client = meilisearch.Client(url = os.getenv('MEILI_SEARCH_URL'),
                            api_key = os.getenv('MEILI_SEARCH_API_KEY'))
client.index('nasdaq').add_documents(result_d, primary_key='id')

TaskInfo(task_uid=4, index_uid='nasdaq', status='enqueued', type='documentAdditionOrUpdate', enqueued_at=datetime.datetime(2025, 5, 27, 8, 40, 28, 14454))

In [30]:
client.index('nasdaq').search('Apple Inc.')

{'hits': [{'id': 'AAPL',
   'Symbol': 'AAPL',
   'Name': 'Apple Inc. Common Stock',
   'Last Sale': '$195.27',
   'Net Change': -6.09,
   '% Change': '-3.024%',
   'Market Cap': '2916518743020.00',
   'Country': 'United States',
   'IPO Year': '1980',
   'Volume': 78313877,
   'Sector': 'Technology',
   'Industry': 'Computer Manufacturing'},
  {'id': 'APLE',
   'Symbol': 'APLE',
   'Name': 'Apple Hospitality REIT Inc. Common Shares',
   'Last Sale': '$11.47',
   'Net Change': -0.11,
   '% Change': '-0.95%',
   'Market Cap': '2728877468.00',
   'Country': 'United States',
   'IPO Year': '2015',
   'Volume': 2427231,
   'Sector': 'Real Estate',
   'Industry': 'Real Estate Investment Trusts'}],
 'query': 'Apple Inc.',
 'processingTimeMs': 2,
 'limit': 20,
 'offset': 0,
 'estimatedTotalHits': 2}